In [24]:
import sys
sys.path.append('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/')
import pandas as pd 
import numpy as np 
import datasets
from datasets import Dataset as TFDataset 
import pickle
from bm25.rank_bm25 import BM25Okapi
import random
# import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
from transformers import PreTrainedTokenizer, AutoTokenizer
import multiprocessing

In [25]:
def min_max_normalization(score_df, score_name, use_zero=True):
    # score_df_g = cudf.from_pandas(score_df)
    if not use_zero:
        score_df_ = score_df[score_df[score_name] != 0]
    else:
        score_df_ = score_df
    scores_min = score_df_[['sess_id', score_name]].groupby('sess_id')[score_name].min().to_numpy()
    scores_max = score_df_[['sess_id', score_name]].groupby('sess_id')[score_name].max().to_numpy()

    score_df['scores_min'] = scores_min[score_df['sess_id']]
    score_df['scores_max'] = scores_max[score_df['sess_id']]

    # merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df['min_max_'+score_name] = (score_df[score_name] - score_df['scores_min']) / (score_df['scores_max'] - score_df['scores_min'])
    score_df.drop(columns=['scores_min', 'scores_max'], inplace=True)

    if not use_zero:
        score_df['min_max_'+score_name][score_df[score_name] == 0] = 0.0


    # del scores_sum_g
    # del merged_score_df_g 

In [26]:
def cast_dtype(df : pd.DataFrame, columns=None):
    if columns is None:
        columns = df.columns
    for k in columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

# Merge valid scores

In [27]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_feature.parquet'
product_data_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions_phase2.csv'

In [28]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(product_data_path)

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)


In [29]:
merged_candidates_feature = read_merged_candidates_feature()

In [30]:
min_max_score_list = ['title_BM25_scores', 'desc_BM25_scores', 'feat_BM25_scores', 'title_bert_scores', 'text_bert_scores']

In [31]:
for score_name in min_max_score_list:
    score_df = merged_candidates_feature[['sess_id', score_name]]
    min_max_normalization(score_df, score_name, use_zero=False)
    merged_candidates_feature['min_max_'+score_name] = score_df['min_max_'+score_name]

/tmp/ipykernel_32972/1800093031.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['scores_min'] = scores_min[score_df['sess_id']]
/tmp/ipykernel_32972/1800093031.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_df['scores_max'] = scores_max[score_df['sess_id']]
/tmp/ipykernel_32972/1800093031.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [36]:
merged_candidates_feature

,sess_id,sess_locale,product,target,sess_avg_price,product_price,sasrec_scores_3,normalized_sasrec_scores_3,sasrec_scores_2,normalized_sasrec_scores_2,...,feat_BM25_scores,gru4rec_cat_scores_2,normalized_gru4rec_cat_scores_2,narm_cat_scores,normalized_narm_cat_scores,min_max_feat_BM25_scores,min_max_title_BM25_scores,min_max_desc_BM25_scores,min_max_title_bert_scores,min_max_text_bert_scores
0,0,DE,355165591X,0.0,43.256542,8.990000,2.230508,7.658405e-09,0.512931,1.377575e-09,...,0.000000,6.108637,4.470486e-11,12.679722,8.364829e-15,0.000000,0.000000,0.000762,0.153855,0.286664
1,0,DE,3833237058,0.0,43.256542,22.000000,9.605231,1.221631e-05,9.325538,9.255110e-06,...,0.000000,10.445271,3.417679e-09,15.522545,1.435752e-13,0.000000,0.216290,0.001514,0.732791,0.706660
2,0,DE,B00CIXSI6U,0.0,43.256542,6.470000,0.714114,1.681035e-09,-0.115904,7.345399e-10,...,0.000000,7.719980,2.239505e-10,10.980855,1.529849e-15,0.000000,0.000000,0.000000,0.042145,0.129999
3,0,DE,B00NVDOWUW,0.0,43.256542,11.990000,8.750996,5.199363e-06,8.507557,4.084482e-06,...,124.140724,11.026890,6.113992e-09,17.282921,8.348356e-13,0.767795,0.434127,0.422333,0.698845,0.620177
4,0,DE,B00NVDP3ZU,0.0,43.256542,22.990000,8.056712,2.596729e-06,5.898870,3.007453e-07,...,104.910095,13.295679,5.910817e-08,15.589283,1.534841e-13,0.648856,0.358621,0.365976,0.633025,0.606537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78842194,261815,UK,B0BCX524Y6,0.0,9.383333,16.990000,6.813615,1.076201e-03,7.203015,4.597607e-04,...,0.000000,8.465229,4.723617e-04,7.993954,7.532316e-04,0.000000,0.193631,0.053563,0.659066,0.654652
78842195,261815,UK,B0BCX6QB4L,0.0,9.383333,10.990000,9.030836,9.881445e-03,10.123234,8.526421e-03,...,0.000000,11.630428,1.119189e-02,10.798326,1.244090e-02,0.000000,0.193631,0.053563,0.659066,0.654652
78842196,261815,UK,B0BFPJYXQL,0.0,9.383333,10.560000,0.796892,2.623396e-06,1.711608,1.895152e-06,...,84.137230,4.747618,1.147458e-05,5.601507,6.884967e-05,0.583092,0.360852,0.310018,0.360322,0.205679
78842197,261815,UK,B0BH3X67S3,0.0,9.383333,6.830000,4.250781,8.296004e-05,6.447586,2.159998e-04,...,31.881231,7.545274,1.882536e-04,6.008080,1.033891e-04,0.220945,0.068835,0.102401,0.441689,0.403038


In [37]:
score_name_list = ['min_max_'+score_name for score_name in min_max_score_list]
cast_dtype(merged_candidates_feature, score_name_list)
merged_candidates_feature.to_parquet(merged_candidates_feature_path, engine='pyarrow')

In [38]:
merged_candidates_feature['min_max_text_bert_scores']

0           0.286664
1           0.706660
2           0.129999
3           0.620177
4           0.606537
              ...   
78842194    0.654652
78842195    0.654652
78842196    0.205679
78842197    0.403038
78842198    0.339988
Name: min_max_text_bert_scores, Length: 78842199, dtype: float32

In [41]:
merged_candidates_min_max_BM25_Bert = merged_candidates_feature[['sess_id', 'product',
                           'min_max_title_BM25_scores', 'min_max_desc_BM25_scores', 'min_max_feat_BM25_scores', 
                           'min_max_title_bert_scores', 'min_max_text_bert_scores']]

In [42]:
merged_candidates_min_max_BM25_Bert

,sess_id,product,min_max_title_BM25_scores,min_max_desc_BM25_scores,min_max_feat_BM25_scores,min_max_title_bert_scores,min_max_text_bert_scores
0,0,355165591X,0.000000,0.000762,0.000000,0.153855,0.286664
1,0,3833237058,0.216290,0.001514,0.000000,0.732791,0.706660
2,0,B00CIXSI6U,0.000000,0.000000,0.000000,0.042145,0.129999
3,0,B00NVDOWUW,0.434127,0.422333,0.767795,0.698845,0.620177
4,0,B00NVDP3ZU,0.358621,0.365976,0.648856,0.633025,0.606537
...,...,...,...,...,...,...,...
78842194,261815,B0BCX524Y6,0.193631,0.053563,0.000000,0.659066,0.654652
78842195,261815,B0BCX6QB4L,0.193631,0.053563,0.000000,0.659066,0.654652
78842196,261815,B0BFPJYXQL,0.360852,0.310018,0.583092,0.360322,0.205679
78842197,261815,B0BH3X67S3,0.068835,0.102401,0.220945,0.441689,0.403038


In [ ]:
merged_candidates_min_max_BM25_Bert.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/transpose/cxl_merged_candidates_min_max_BM25_Bert_scores.parquet', engine='pyarrow')